In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('train.txt',sep=';',header=None,names=['text','emotion'])

In [3]:
df.head()

,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [4]:
df.isnull()

,text,emotion
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
15995,False,False
15996,False,False
15997,False,False
15998,False,False


In [5]:
df.isnull().sum()

,0
text,0
emotion,0


In [6]:
unique_emotions=df['emotion'].unique()
emotion_numbers={}
i=0
for emo in unique_emotions:
  emotion_numbers[emo]=i
  i+=1
df['emotion']=df['emotion'].map(emotion_numbers)

In [7]:
df

,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [8]:
df['text']=df['text'].apply(lambda x:x.lower())

In [9]:
import string
def remove_punctuation(txt):
  return txt.translate(str.maketrans('','',string.punctuation))

In [10]:
df['text']=df['text'].apply(remove_punctuation)

In [11]:
def remove_numbers(txt):
  new=""
  for i in txt:
    if not i.isdigit():
      new=new+i
  return new

df['text']=df['text'].apply(remove_numbers)

In [12]:
"""Removing links"""
"""import re

def remove_urls(text):
    url_pattern = r'http\S+|www\.\S+'   # matches http://, https://, www.
    return re.sub(url_pattern, '', text)

df['text'] = df['text'].apply(remove_urls)
"""

<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:5: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-16017804.py:5: SyntaxWarning: invalid escape sequence '\S'
  url_pattern = r'http\S+|www\.\S+'   # matches http://, https://, www.


"import re\n\ndef remove_urls(text):\n    url_pattern = r'http\\S+|www\\.\\S+'   # matches http://, https://, www.\n    return re.sub(url_pattern, '', text)\n\ndf['text'] = df['text'].apply(remove_urls)\n"

In [13]:
def remove_emojis(txt):
  new=""
  for i in txt:
    if i.isascii():
      new+=i
  return new
df['text']=df['text'].apply(remove_emojis)

In [14]:
import nltk

In [15]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [16]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stop_words=set(stopwords.words('english'))


In [18]:
df.loc[1]

,1
text,i can go from feeling so hopeless to so damned...
emotion,0


In [19]:
df

,text,emotion
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,1
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,1
...,...,...
15995,i just had a very brief time in the beanbag an...,0
15996,i am now turning and i feel pathetic that i am...,0
15997,i feel strong and good overall,5
15998,i feel like this was such a rude comment and i...,1


In [20]:
df.loc[1]['text']

'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake'

In [21]:
def remove(txt):
  words=txt.split()
  cleaned=[]
  for i in words:
    if i not in stop_words:
      cleaned.append(i)
  return " ".join(cleaned)

In [22]:
df['text']=df['text'].apply(remove)

In [23]:
df.loc[1]['text']

'go feeling hopeless damned hopeful around someone cares awake'

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['emotion'], test_size=0.20, random_state=42)


In [25]:
X_train

,text
676,refers course though cant help feeling somehow...
12113,im starting feel im suffering fatigue
7077,feel like probably would liked book little bit...
13005,really feel awkward
12123,im feeling little grumpy today lame weather te...
...,...
13418,love leave reader feeling confused slightly de...
5390,feel delicate
860,starting feel little stressed
15795,feel stressed tired worn shape neglected


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

bow_vectorizer=CountVectorizer()

X_train_bow=bow_vectorizer.fit_transform(X_train)
X_test_bow=bow_vectorizer.transform(X_test)

nb_model=MultinomialNB()
nb_model.fit(X_train_bow,y_train)

pred_bow=nb_model.predict(X_test_bow)
print(accuracy_score(y_test,pred_bow))

0.768125


In [27]:
pred_bow

array([0, 5, 0, ..., 5, 5, 0])

In [28]:
y_test

,emotion
8756,0
4660,5
6095,0
304,5
8241,0
...,...
15578,5
5746,5
6395,5
7624,5


In [30]:
tfidf_vectorizer=TfidfVectorizer()

X_train_tfidf=tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf=tfidf_vectorizer.transform(X_test)

nb2_model=MultinomialNB()
nb2_model.fit(X_train_bow,y_train)



MultinomialNB()

In [31]:
y_pred=nb2_model.predict(X_test_tfidf)
print(accuracy_score(y_test,y_pred))

0.7275


In [32]:
from sklearn.linear_model import LogisticRegression


In [33]:
logistic_model=LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_tfidf,y_train)

LogisticRegression(max_iter=1000)

In [34]:
log_pred=logistic_model.predict(X_test_tfidf)
print(accuracy_score(y_test,log_pred))

0.8628125
